In [1]:
# --- Step 1.1: Setup and Ingestion (Safe Version) ---

# 1. Install required libraries
print("Installing libraries... (This may take a minute)")
!pip install moviepy -q
!pip install scenedetect -q
!pip install opencv-python -q
print("✅ Libraries installed successfully.")

# 2. Import all necessary libraries for the project
import os
import glob
import cv2
import numpy as np
import pandas as pd
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip

# Now this import will work!
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
# --- FIX ---
# Removed all specific function imports from scene_manager
# as they are not needed for Step 1.2
# --- END FIX ---
from tqdm.notebook import tqdm # For a nice progress bar

print("✅ All libraries imported.")

# 3. Define file paths
INPUT_DATASET_PATH = "/kaggle/input/short-clips/dataset1/"
OUTPUT_DIR = "/kaggle/working/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. Find all video clips to process
video_files = glob.glob(os.path.join(INPUT_DATASET_PATH, "*.mp4"))
video_files.sort() # Sort them to process in a consistent order

# 5. Confirmation
if video_files:
    print(f"✅ Found {len(video_files)} video files to process:")
    for f in video_files:
        print(f"  - {os.path.basename(f)}")
else:
    print(f"⚠️ Warning: No .mp4 files found in {INPUT_DATASET_PATH}")

Installing libraries... (This may take a minute)
✅ Libraries installed successfully.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

AttributeError: _ARRAY_API not found

error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings

✅ All libraries imported.
✅ Found 13 video files to process:
  - asconvpt.mp4
  - blostjdq.mp4
  - chqblpze.mp4
  - frbnwkkq.mp4
  - kjhjjxni.mp4
  - myqiuzzc.mp4
  - ncikuizq.mp4
  - ohwchuju.mp4
  - ukonzgxq.mp4
  - unygqzdu.mp4
  - wcsdambp.mp4
  - wuqoarpl.mp4
  - wylqhkno.mp4



ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default


In [2]:
# --- Step 1.2: Shot Detection & Analysis Pipeline ---

# This list will hold all the data about every shot
all_shots_data = []

def find_shots(video_path):
    """
    Uses PySceneDetect to find all the shots in a single video file.
    Returns a list of (start_frame, end_frame) tuples.
    """
    try:
        video_manager = VideoManager([video_path])
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=27.0))
        
        # This part does the work
        video_manager.set_downscale_factor(1) # Process at full resolution
        video_manager.start()
        scene_manager.detect_scenes(frame_source=video_manager)
        
        # Get the list of scenes (shots)
        scene_list = scene_manager.get_scene_list()
        
        # Handle case where no scenes are detected (the whole clip is one shot)
        if not scene_list:
            # Get total frames from the video_manager
            total_frames = int(video_manager.get_duration().get_frames())
            if total_frames > 0:
                return [(video_manager.get_framerate().get_timecode(0), 
                         video_manager.get_duration())]
            else:
                return []
                
        video_manager.release()
        return scene_list
        
    except Exception as e:
        print(f"Error processing {video_path}: {e}")
        return []

def get_blur_score(frame):
    """
    Calculates the blurriness of a single frame using Laplacian variance.
    A higher score is SHARPER. A lower score is BLURRIER.
    """
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Calculate the variance of the Laplacian
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance

# --- Main Processing Loop ---
# This will go through every video file one by one.
print(f"Starting analysis of {len(video_files)} files...")

for video_path in tqdm(video_files, desc="Processing Videos"):
    video_filename = os.path.basename(video_path)
    print(f"\nProcessing: {video_filename}")
    
    shots = find_shots(video_path)
    
    if not shots:
        print(f"  -> No shots found, skipping.")
        continue
        
    print(f"  -> Found {len(shots)} shots.")
    
    # Now we analyze each shot in this video
    cap = cv2.VideoCapture(video_path)
    
    for i, (start_time, end_time) in enumerate(tqdm(shots, desc="Analyzing Shots")):
        start_frame = start_time.get_frames()
        end_frame = end_time.get_frames()
        
        # Get the middle frame of the shot for analysis
        # This is more efficient than analyzing every single frame
        middle_frame_num = int((start_frame + end_frame) / 2)
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_num)
        
        ret, frame = cap.read()
        
        if ret and frame is not None:
            # --- Analyze the frame ---
            blur_score = get_blur_score(frame)
            
            # --- Store the results ---
            shot_data = {
                'video_file': video_filename,
                'video_path': video_path,
                'shot_num': i,
                'start_timecode': start_time.get_timecode(),
                'end_timecode': end_time.get_timecode(),
                'duration_sec': (end_frame - start_frame) / start_time.framerate,
                'blur_score': blur_score
                # We will add 'face_score', 'shake_score', etc. here
            }
            all_shots_data.append(shot_data)
        
    cap.release()

print("\n--- Analysis Complete ---")

# --- Create the DataFrame ---
# This is our key "pipeline" artifact
df_shots = pd.DataFrame(all_shots_data)

# Let's look at the results
print(f"✅ Created DataFrame with {len(df_shots)} total shots.")
if not df_shots.empty:
    print(df_shots.head())
    
    # Let's see the "blurriest" shots we found as a test
    print("\nBlurriest shots found:")
    print(df_shots.sort_values(by='blur_score').head())
    
    # Let's see the "sharpest" shots we found
    print("\nSharpest shots found:")
    print(df_shots.sort_values(by='blur_score', ascending=False).head())
else:
    print("⚠️ Warning: The DataFrame is empty. No shots were analyzed.")

Starting analysis of 13 files...


Processing Videos:   0%|          | 0/13 [00:00<?, ?it/s]


Processing: asconvpt.mp4
  -> Found 4 shots.


Analyzing Shots:   0%|          | 0/4 [00:00<?, ?it/s]


Processing: blostjdq.mp4
  -> Found 6 shots.


...


Processing: wuqoarpl.mp4
  -> Found 16 shots.


Analyzing Shots:   0%|          | 0/16 [00:00<?, ?it/s]


Processing: wylqhkno.mp4
  -> Found 4 shots.


Analyzing Shots:   0%|          | 0/4 [00:00<?, ?it/s]


--- Analysis Complete ---
✅ Created DataFrame with 79 total shots.
     video_file                                       video_path  shot_num  \
0  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         0   
1  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         1   
2  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         2   
3  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         3   
4  blostjdq.mp4  /kaggle/input/short-clips/dataset1/blostjdq.mp4         0   

  start_timecode  end_timecode  duration_sec  blur_score  
0   00:00:00.000  00:00:20.760         20.76   64.161774  
1   00:00:20.760  00:00:59.300         38.54   89.137438  
2   00:00:59.300  00:01:00.940          1.64   84.508437  
3   00:01:00.940  00:01:01.880          0.94  202.184176  
4   00:00:00.000  00:00:03.620          3.62  146.572058  

Blurriest shots found:
      video_file                                       video_path  shot_num  \
37  oh

In [3]:
# --- Step 1.3: Advanced Shot Analysis (Blur, Faces, Motion) ---
# We already have the 'df_shots' DataFrame from the previous cell,
# but we will rebuild it with all the new scores.

# --- 1. Load Face Detection Model ---
# This path points to the pre-installed file in the Kaggle environment
face_cascade_path = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier()

if not face_cascade.load(face_cascade_path):
    print("⚠️ Warning: Could not load face cascade classifier.")
else:
    print("✅ Face detection model loaded successfully.")

# --- 2. Define Analysis Functions ---

def get_blur_score(frame):
    """
    Calculates the blurriness of a single frame using Laplacian variance.
    A higher score is SHARPER. A lower score is BLURRIER.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    variance = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance

def get_face_score(frame, gray_frame):
    """
    Calculates a score based on the presence and size of faces.
    Score is the total pixel area of all detected face bounding boxes.
    """
    faces = face_cascade.detectMultiScale(
        gray_frame, 
        scaleFactor=1.1, 
        minNeighbors=5, 
        minSize=(30, 30)
    )
    
    total_face_area = 0
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            total_face_area += w * h
    return total_face_area

def get_motion_score(prev_gray, next_gray):
    """
    Calculates a motion score using optical flow.
    A higher score means more movement/action (or shakiness).
    """
    flow = cv2.calcOpticalFlowFarneback(
        prev_gray, 
        next_gray, 
        None, 
        0.5, # pyr_scale
        3,   # levels
        15,  # winsize
        3,   # iterations
        5,   # poly_n
        1.2, # poly_sigma
        0    # flags
    )
    
    # Calculate the magnitude (length) of the 2D flow vectors
    magnitude, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    
    # Return the average magnitude across the whole frame
    return np.mean(magnitude)


# --- 3. Main Processing Loop ---

print(f"\nStarting advanced analysis of {len(video_files)} files...")
all_shots_data = [] # Reset the list

for video_path in tqdm(video_files, desc="Processing Videos"):
    video_filename = os.path.basename(video_path)
    
    # This function is from your previous cell
    shots = find_shots(video_path) 
    
    if not shots:
        continue
        
    cap = cv2.VideoCapture(video_path)
    
    for i, (start_time, end_time) in enumerate(shots):
        start_frame = start_time.get_frames()
        end_frame = end_time.get_frames()
        
        # We need two consecutive frames for motion analysis
        middle_frame_num = int((start_frame + end_frame) / 2)
        
        # Read middle frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_num)
        ret_mid, frame_mid = cap.read()
        
        # Read the frame right after it
        cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame_num + 1)
        ret_next, frame_next = cap.read()
        
        if ret_mid and ret_next and frame_mid is not None and frame_next is not None:
            # Convert to grayscale
            gray_mid = cv2.cvtColor(frame_mid, cv2.COLOR_BGR2GRAY)
            gray_next = cv2.cvtColor(frame_next, cv2.COLOR_BGR2GRAY)
            
            # --- Analyze the frames ---
            blur_score = get_blur_score(frame_mid)
            face_score = get_face_score(frame_mid, gray_mid)
            motion_score = get_motion_score(gray_mid, gray_next)
            
            # --- Store the results ---
            shot_data = {
                'video_file': video_filename,
                'video_path': video_path,
                'shot_num': i,
                'start_timecode': start_time.get_timecode(),
                'end_timecode': end_time.get_timecode(),
                'duration_sec': (end_frame - start_frame) / start_time.framerate,
                'blur_score': blur_score,
                'face_score': face_score,
                'motion_score': motion_score
            }
            all_shots_data.append(shot_data)
        
    cap.release()

print("\n--- Advanced Analysis Complete ---")

# --- 4. Create the Final DataFrame ---
df_shots = pd.DataFrame(all_shots_data)

if not df_shots.empty:
    print(f"✅ Created DataFrame with {len(df_shots)} total shots.")
    
    # Save the DataFrame to a CSV file for easy re-use
    # This is a key part of our "pipeline"!
    df_shots.to_csv(os.path.join(OUTPUT_DIR, "shot_analysis_results.csv"), index=False)
    print(f"✅ Analysis data saved to {OUTPUT_DIR}shot_analysis_results.csv")

    # Let's look at the results
    print("\n--- Sample of Analysis Data ---")
    print(df_shots.head())
    
    print("\n--- Top 5 'Best Face' Shots ---")
    print(df_shots.sort_values(by='face_score', ascending=False).head())
    
    print("\n--- Top 5 'Highest Motion' Shots ---")
    print(df_shots.sort_values(by='motion_score', ascending=False).head())
else:
    print("⚠️ Warning: The DataFrame is empty. No shots were analyzed.")

✅ Face detection model loaded successfully.

Starting advanced analysis of 13 files...


Processing Videos:   0%|          | 0/13 [00:00<?, ?it/s]

Error processing /kaggle/input/short-clips/dataset1/frbnwkkq.mp4: 'tuple' object has no attribute 'get_frames'

--- Advanced Analysis Complete ---
✅ Created DataFrame with 79 total shots.
✅ Analysis data saved to /kaggle/working/shot_analysis_results.csv

--- Sample of Analysis Data ---
     video_file                                       video_path  shot_num  \
0  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         0   
1  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         1   
2  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         2   
3  asconvpt.mp4  /kaggle/input/short-clips/dataset1/asconvpt.mp4         3   
4  blostjdq.mp4  /kaggle/input/short-clips/dataset1/blostjdq.mp4         0   

  start_timecode  end_timecode  duration_sec  blur_score  face_score  \
0   00:00:00.000  00:00:20.760         20.76   64.161774        3364   
1   00:00:20.760  00:00:59.300         38.54   89.137438           0   
2   00:00:59.300  00:01:00.

In [4]:
# --- Stage 2: Scoring, Filtering, and Selection ---
import pandas as pd
import numpy as np
import os

# --- 1. Load our Analysis Data ---
# This proves our pipeline is modular. We don't have to re-run Stage 1.
DATA_FILE = os.path.join(OUTPUT_DIR, "shot_analysis_results.csv")

if not os.path.exists(DATA_FILE):
    print("⚠️ Error: shot_analysis_results.csv not found!")
    print("Please re-run the previous analysis cell (Step 1.3).")
else:
    print(f"✅ Successfully loaded {DATA_FILE}")
    df_shots = pd.read_csv(DATA_FILE)

# --- 2. Normalize the Data (Feature Scaling) ---
# We scale all scores from 0 to 1 so we can combine them fairly.
# (e.g., a face_score of 90k isn't 90,000x more important than a blur_score of 100)
df_shots['blur_norm'] = (df_shots['blur_score'] - df_shots['blur_score'].min()) / (df_shots['blur_score'].max() - df_shots['blur_score'].min())
df_shots['face_norm'] = (df_shots['face_score'] - df_shots['face_score'].min()) / (df_shots['face_score'].max() - df_shots['face_score'].min())
df_shots['motion_norm'] = (df_shots['motion_score'] - df_shots['motion_score'].min()) / (df_shots['motion_score'].max() - df_shots['motion_score'].min())

print("✅ Data normalized (scaled from 0 to 1).")


# --- 3. Define the "Interest Score" Algorithm ---
# This is the "secret sauce" for the "technical depth" criterion.
# We can tune these weights to change the style of the final video.
W_FACE = 1.5   # We really care about faces
W_SHARPNESS = 1.0   # We like sharp shots
W_MOTION = 0.5   # We like *some* motion, but it's less important
W_DURATION = 0.1 # We slightly prefer longer shots over 1-second cuts

def calculate_interest_score(row):
    score = (row['face_norm'] * W_FACE) \
          + (row['blur_norm'] * W_SHARPNESS) \
          + (row['motion_norm'] * W_MOTION) \
          + (np.log1p(row['duration_sec']) * W_DURATION) # Use log for duration
    return score

df_shots['interest_score'] = df_shots.apply(calculate_interest_score, axis=1)
print("✅ 'interest_score' calculated.")


# --- 4. Filter Out "Bad" Shots ---
# These are hard-coded rules to ensure "Quality" (a judging criterion).
# We will REMOVE any shot that...
MIN_BLUR_THRESHOLD = 50.0  # Is too blurry (based on our 1.2 output)
MIN_DURATION_SEC = 1.5     # Is too short to be cinematic
MAX_MOTION_SCORE = 3.0     # Is *too* shaky (based on our 1.3 output)

# Create a 'rejection_reason' column to explain our logic
def get_rejection_reason(row):
    if row['blur_score'] < MIN_BLUR_THRESHOLD:
        return "Too Blurry"
    if row['duration_sec'] < MIN_DURATION_SEC:
        return "Too Short"
    if row['motion_score'] > MAX_MOTION_SCORE:
        return "Too Shaky"
    return None # Keep this shot

df_shots['rejection_reason'] = df_shots.apply(get_rejection_reason, axis=1)

# Create our final list of "good" shots to choose from
df_finalist_shots = df_shots[df_shots['rejection_reason'].isnull()].copy()
df_rejected_shots = df_shots[df_shots['rejection_reason'].notnull()].copy()

print(f"✅ Filtering complete.")
print(f"  -> {len(df_shots)} total shots analyzed.")
print(f"  -> {len(df_rejected_shots)} shots rejected.")
print(f"  -> {len(df_finalist_shots)} 'good' shots remaining.")


# --- 5. Select the "Best" Shots ---
# We sort by our score and pick the top ones until we hit our time limit.
# (Hackathon requirement: 5-10 minutes)
TARGET_DURATION_SEC = 8 * 60  # Let's aim for 8 minutes

# Sort by best score
df_finalist_shots = df_finalist_shots.sort_values(by='interest_score', ascending=False)

# Add shots to our "final edit list" one by one
selected_shots = []
total_duration = 0
for index, row in df_finalist_shots.iterrows():
    if total_duration + row['duration_sec'] <= TARGET_DURATION_SEC:
        selected_shots.append(row)
        total_duration += row['duration_sec']

df_edit_list = pd.DataFrame(selected_shots)

print(f"\n--- Final Selection Complete ---")
print(f"✅ Selected {len(df_edit_list)} shots.")
print(f"✅ Final video duration: {total_duration / 60:.2f} minutes.")

print("\n--- Top 10 Shots Selected for Final Video ---")
print(df_edit_list[['video_file', 'duration_sec', 'interest_score', 'face_score', 'blur_score', 'motion_score']].head(10))

print("\n--- Top 5 Shots that were REJECTED ---")
print(df_rejected_shots[['video_file', 'duration_sec', 'rejection_reason', 'blur_score', 'motion_score']].head())

✅ Successfully loaded /kaggle/working/shot_analysis_results.csv
✅ Data normalized (scaled from 0 to 1).
✅ 'interest_score' calculated.
✅ Filtering complete.
  -> 79 total shots analyzed.
  -> 28 shots rejected.
  -> 51 'good' shots remaining.

--- Final Selection Complete ---
✅ Selected 50 shots.
✅ Final video duration: 7.74 minutes.

--- Top 10 Shots Selected for Final Video ---
      video_file  duration_sec  interest_score  face_score  blur_score  \
48  wcsdambp.mp4          3.60        2.145333      101964  663.589515   
47  wcsdambp.mp4          3.78        2.093474       91792  711.802823   
45  unygqzdu.mp4          2.14        1.685162      121481   65.858386   
71  wuqoarpl.mp4         16.28        1.671743       59693  558.334193   
41  ukonzgxq.mp4         32.42        1.641155       68046  381.143520   
73  wuqoarpl.mp4          3.34        1.631759       61876  556.101346   
16  chqblpze.mp4         31.34        1.611682       79979   83.400410   
38  ohwchuju.mp4         

In [7]:
# --- Stage 3 & 4: Audio Upload and Final Video Assembly (CORRECTED) ---

# --- 1. Import all required libraries ---
from moviepy.editor import (VideoFileClip, concatenate_videoclips, AudioFileClip, 
                            CompositeVideoClip)
import moviepy.video.fx.all as vfx
from moviepy.audio.fx.all import audio_loop
import os
from tqdm.notebook import tqdm

# --- 3.1: Load Your Uploaded Music ---

# --- FIX 1: Using the correct path from your Dataset ---
MUSIC_FILE = "/kaggle/input/musicc/my-cool-music.mp3"
# --- END FIX ---

music = None
if not os.path.exists(MUSIC_FILE):
    print(f"⚠️ Music file not found at {MUSIC_FILE}")
    print("Please double-check the 'musicc' dataset name.")
else:
    try:
        music = AudioFileClip(MUSIC_FILE)
        print(f"✅ Successfully loaded music: {MUSIC_FILE}")
    except Exception as e:
        print(f"⚠️ Could not load audio file: {e}. The final video will be silent.")


# --- 4.1: Assemble Final Video Clips ---
print("\n--- Starting Final Video Assembly ---")
print(f"Loading and trimming {len(df_edit_list)} selected shots...")

TRANSITION_SEC = 0.5 
final_clips_list = []

# --- FIX 2: Fixed the 'NoneType' bug ---
# We will keep parent clips in this list to close them LATER
parent_clips_to_close = []
# --- END FIX ---

for index, row in tqdm(df_edit_list.iterrows(), total=len(df_edit_list)):
    try:
        clip = VideoFileClip(row['video_path'])
        parent_clips_to_close.append(clip) # Add parent clip to our new list
        
        clip_trimmed = clip.subclip(row['start_timecode'], row['end_timecode'])
        clip_trimmed = clip_trimmed.set_audio(None)
        clip_with_fade = clip_trimmed.fx(vfx.fadein, TRANSITION_SEC)
        
        final_clips_list.append(clip_with_fade)
        
        # --- REMOVED `clip.close()` FROM HERE ---
        
    except Exception as e:
        print(f"⚠️ Warning: Failed to process shot {row['video_file']} at {row['start_timecode']}. Error: {e}. Skipping.")
        
print(f"✅ {len(final_clips_list)} clips prepared.")


# --- 4.2: Concatenate with Transitions ---
if final_clips_list:
    final_video = concatenate_videoclips(final_clips_list, 
                                         padding = -TRANSITION_SEC, 
                                         method="compose")

    # --- 4.3: Add Background Music ---
    if music:
        print("Adding background music...")
        # Your video is 7.74 min and music is 5.5 min, so we'll loop it.
        if music.duration < final_video.duration:
            print("Music is shorter than video, looping audio...")
            music = audio_loop(music, duration=final_video.duration)
        
        music = music.subclip(0, final_video.duration)
        final_video = final_video.set_audio(music)

    # --- 4.4: Render Final Video ---
    FINAL_VIDEO_PATH = os.path.join(OUTPUT_DIR, "hackathon_submission.mp4")
    print(f"\n--- 🚀 Rendering Final Video ---")
    print(f"This will take several minutes...")
    
    try:
        final_video.write_videofile(FINAL_VIDEO_PATH, 
                                    codec='libx264', 
                                    audio_codec='aac', 
                                    threads=4, 
                                    preset='medium')
        
        print(f"\n🎉🎉🎉 --- SUCCESS! --- 🎉🎉🎉")
        print(f"Your final cinematic video is saved to:")
        print(FINAL_VIDEO_PATH)
        print("You can download it from the 'Output' section on the right-hand panel.")
        
    except Exception as e:
        print(f"🔥🔥🔥 Rendering Failed: {e}")

    # --- 4.5: Final Cleanup ---
    print("Cleaning up file handles...")
    if music:
        music.close()
    for clip in final_clips_list:
        clip.close()
    final_video.close()
    
    # --- FIX 2 (Continued): Now we close the parent clips ---
    for clip in parent_clips_to_close:
        clip.close()
    print("✅ Cleanup complete.")
    
else:
    print("⚠️ No clips were prepared. Final video not created.")

✅ Successfully loaded music: /kaggle/input/musicc/my-cool-music.mp3

--- Starting Final Video Assembly ---
Loading and trimming 50 selected shots...


  0%|          | 0/50 [00:00<?, ?it/s]

✅ 50 clips prepared.
Adding background music...
Music is shorter than video, looping audio...

--- 🚀 Rendering Final Video ---
This will take several minutes...
Moviepy - Building video /kaggle/working/hackathon_submission.mp4.
MoviePy - Writing audio in hackathon_submissionTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video /kaggle/working/hackathon_submission.mp4



Moviepy - Done !
Moviepy - video ready /kaggle/working/hackathon_submission.mp4

🎉🎉🎉 --- SUCCESS! --- 🎉🎉🎉
Your final cinematic video is saved to:
/kaggle/working/hackathon_submission.mp4
You can download it from the 'Output' section on the right-hand panel.
Cleaning up file handles...
✅ Cleanup complete.
